<a href="https://www.kaggle.com/code/srsses/binary-prediction-of-smoker-status?scriptVersionId=156349320" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
train_data = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")
train_data.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0,1


In [3]:
train_data.drop('id', axis = 1, inplace = True)

In [4]:
# train_data['BMI'] = round(train_data['weight(kg)']/((train_data['height(cm)']/100)**2),2)
# train_data['avg_sight'] = round((train_data['eyesight(left)']+train_data['eyesight(right)'])/2,2)
# train_data['hdl_ldl_ratio'] = round(train_data['HDL']/train_data['LDL'],2)
# # train_data['bad_good_chol'] = train_data['LDL'].apply(lambda x : {x <= 100:1,100 < x <= 129:2,130 <= x <= 159:3}.get(x,''))
# train_data['AST_ALT_AVG'] = (train_data['AST']+train_data['ALT'])/2
train_data['serum creatinine'] = train_data['serum creatinine'].apply(lambda x:1/(1+np.exp(-x)))

In [5]:
train_data.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,0.731059,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,0.750260,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.689974,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,0.731059,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.689974,19,13,17,0,1


In [6]:
train_data.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,0.731059,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,0.750260,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.689974,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,0.731059,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.689974,19,13,17,0,1


In [7]:
def remove_outlier(df, cols):
    Q1 = df[cols].quantile(0.15)
    Q3 = df[cols].quantile(0.85)
    df.loc[df[cols] < Q1, cols] = Q1
    df.loc[df[cols] > Q3, cols] = Q3
    df[cols] = np.sqrt(df[cols])

In [8]:
colss = train_data.drop('smoking', axis = 1).columns

In [9]:
for i in colss:
    remove_outlier(train_data.drop('smoking', axis = 1),i)

In [10]:
x = train_data.drop('smoking', axis = 1)
y = train_data['smoking']
import lightgbm

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc  = StandardScaler()
# m = MinMaxScaler()
x = sc.fit_transform(x)

In [12]:
#from lightgbm import LGBMClassifier
#lgb  = LGBMClassifier(max_depth = 44,learning_rate = 0.001,n_estimators = 2500,objective = 'binary')
#lgb.fit(x,y)

In [13]:
from sklearn.metrics import roc_auc_score,f1_score
#print(roc_auc_score(y,lgb.predict(x)))
#f1_score(y,lgb.predict(x))

In [14]:
#lgb.predict(x)

In [15]:
#y

In [16]:
#df = pd.DataFrame({'value':lgb.feature_importances_, 'features':train_data.drop('smoking',axis=1).columns})
#df.sort_values(by = 'value')

In [17]:
#lgb.feature_name_

In [18]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4, random_state = 3,shuffle = True)

In [19]:
 import keras
from keras.models import Sequential
 from keras.layers import Dense, Dropout

In [20]:
 model = Sequential()

In [21]:
model.add(Dense(400,activation = 'relu',kernel_initializer = 'HeNormal'))
model.add(Dense(500,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(600,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [22]:
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),loss = 'binary_crossentropy', metrics = 'AUC')

In [23]:
model.fit(x,y,batch_size = 1024,epochs = 5)

Epoch 1/5
156/156 [==============================] - 10s 51ms/step - loss: 0.4757 - auc: 0.8414
Epoch 2/5
156/156 [==============================] - 9s 55ms/step - loss: 0.4601 - auc: 0.8532
Epoch 3/5
156/156 [==============================] - 8s 51ms/step - loss: 0.4558 - auc: 0.8563
Epoch 4/5
156/156 [==============================] - 8s 51ms/step - loss: 0.4534 - auc: 0.8581
Epoch 5/5
156/156 [==============================] - 8s 52ms/step - loss: 0.4520 - auc: 0.8591


In [24]:
#print(f1_score(y,model.predict(x)))
print(roc_auc_score(y,model.predict(x)))

4977/4977 [==============================] - 16s 3ms/step
0.8642634206383351


In [25]:
test_data = pd.read_csv("/kaggle/input/playground-series-s3e24/test.csv")
test_data.drop(['id'], axis = 1, inplace = True)

In [26]:
# test_data['BMI'] = round(test_data['weight(kg)']/((test_data['height(cm)']/100)**2),2)
# test_data['avg_sight'] = round((test_data['eyesight(left)']+test_data['eyesight(right)'])/2,2)
# test_data['hdl_ldl_ratio'] = round(test_data['HDL']/test_data['LDL'],2)
# test_data['AST_ALT_AVG'] = (test_data['AST']+test_data['ALT'])/2
test_data['serum creatinine'] = test_data['serum creatinine'].apply(lambda x:1/(1+np.exp(-x)))


for i in colss:
    remove_outlier(test_data,i)

In [27]:
#test_data.drop(['weight(kg)','eyesight(right)','LDL','AST'], axis = 1, inplace = True)

In [28]:
# a = pf.transform(test_data)

In [29]:
test_data = sc.transform(test_data)

In [30]:
test_data

array([[ -3.207336  , -17.28344437,  -4.66997622, ...,  -1.21389505,
         -0.97931729,  -0.49686626],
       [ -3.08730716, -17.30568308,  -4.71928702, ...,  -1.28477556,
         -1.00360461,  -0.49686626],
       [ -3.08730716, -17.26154013,  -4.66997622, ...,  -1.13928489,
         -0.96832063,  -0.49686626],
       ...,
       [ -3.24182907, -17.26154013,  -4.62407885, ...,  -1.17689546,
         -1.00360461,   2.01261403],
       [ -3.207336  , -17.30568308,  -4.71928702, ...,  -1.15756659,
         -0.97931729,  -0.49686626],
       [ -3.14429793, -17.32827208,  -4.74548789, ...,  -1.28477556,
         -1.02846874,   2.01261403]])

In [31]:
pred = model.predict(test_data)

3318/3318 [==============================] - 11s 3ms/step


In [32]:
#pred['prediction_label']

In [33]:
sample = pd.read_csv("/kaggle/input/playground-series-s3e24/sample_submission.csv")
sample.head()

,id,smoking
0,159256,0.5
1,159257,0.5
2,159258,0.5
3,159259,0.5
4,159260,0.5


In [34]:
pred

array([[2.1306215e-09],
       [2.1976259e-09],
       [2.5638314e-09],
       ...,
       [2.6463818e-09],
       [1.2675604e-09],
       [1.7776425e-09]], dtype=float32)

In [35]:
sample['smoking'] = pred

In [36]:
sample

,id,smoking
0,159256,2.130621e-09
1,159257,2.197626e-09
2,159258,2.563831e-09
3,159259,1.881042e-09
4,159260,3.518219e-09
...,...,...
106166,265422,2.739108e-09
106167,265423,2.888052e-09
106168,265424,2.646382e-09
106169,265425,1.267560e-09


In [37]:
sample.to_csv('submission.csv',index= False)